In [1]:
import pandas as pd
import numpy as np
import pandas_plink as pdp


In [22]:
CHROM = 17
VALS_TO_GO_BACK = [100,500,1000]
CHR_FILE_PREFIX = "/sci/archive/michall/roeizucker/standart_genetics/chr"
CHR_FILE_SUFFIX = "_filtered_mafe-6_hwee-5_geno_02"
ATTRIBUTES_FILE = "./wgEncodeGencodeAttrsV12.tab"
LOCS_FILE = "./wgEncodeGencodeBasicV12.gp"

In [23]:
def work_for_chr(chrom):
    pass
work_for_chr(CHROM)

In [29]:
att_df = pd.read_csv(ATTRIBUTES_FILE,sep="\t")
att_df = att_df[att_df["geneType"]=="protein_coding"]
transcripts_to_use = list(set(att_df["transcriptId"]))
len(transcripts_to_use)

139204

In [102]:
# locs_df = pd.read_csv(LOCS_FILE,sep="\t",header=None)
# locs_df = locs_df[locs_df[0].isin(transcripts_to_use)]
# locs_df.drop_duplicates(subset=0, keep="first")

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
4,ENST00000417324.1,chr1,-,34553,36081,35137,35736,3,"34553,35276,35720,","35174,35481,36081,",0,FAM138A,cmpl,cmpl,"2,1,0,"
5,ENST00000461467.1,chr1,-,35244,36073,36073,36073,2,"35244,35720,","35481,36073,",0,FAM138A,none,none,"-1,-1,"
6,ENST00000335137.3,chr1,+,69090,70008,69090,70008,1,"69090,","70008,",0,OR4F5,cmpl,cmpl,"0,"
24,ENST00000426406.1,chr1,+,367639,368634,367658,368597,1,"367639,","368634,",0,OR4F29,cmpl,cmpl,"0,"
35,ENST00000332831.2,chr1,-,621058,622053,621095,622034,1,"621058,","622053,",0,OR4F16,cmpl,cmpl,"0,"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94846,ENST00000382287.1,chrY,-,27177047,27198251,27184244,27190170,9,"27177047,27177463,27184138,27184955,27187915,2...","27177369,27177588,27184263,27185061,27188033,2...",0,BPY2C,cmpl,cmpl,"-1,-1,2,1,0,0,-1,-1,-1,"
94852,ENST00000361963.2,chrY,+,27768263,27770483,27768589,27770212,1,"27768263,","27770483,",0,CDY1,cmpl,cmpl,"0,"
94853,ENST00000306609.4,chrY,+,27768308,27771049,27768589,27770674,2,"27768308,27770601,","27770181,27771049,",0,CDY1,cmpl,cmpl,"0,2,"
94857,ENST00000306589.6,chrY,+,28121643,28134783,28121695,28134216,3,"28121643,28124350,28133957,","28121764,28124466,28134783,",0,PRYP4,cmpl,cmpl,"0,0,2,"


In [119]:
def create_intervals(row):
    start = row[8][:-1].split(",")
    end = row[9][:-1].split(",")
    res = []
    for i in range(len(start)):
        if row[2] == "+":
            if i == 0:
                res.append(f"{i}_({int(start[i]) -75}-{int(end[i])})")
                continue
            res.append(f"{i}_({int(start[i]) -5}-{int(end[i]) + 15})")
        else:
            if i == len(start) - 1:
                res.append(f"{len(start) - i - 1}_({int(start[i]) }-{int(end[i]) + 75})")
                continue
            res.append(f"{len(start) - i - 1}_({int(start[i]) -15}-{int(end[i]) + 5})")
    return (res)
    
locs_df = pd.read_csv(LOCS_FILE,sep="\t",header=None)
locs_df = locs_df.drop_duplicates(subset=11, keep="first")
locs_df = locs_df[locs_df[0].isin(transcripts_to_use)]
# locs_df = locs_df[locs_df[1] == "chr" + str(CHROM)]
# locs_df
locs_df["intervals"] = locs_df.apply(create_intervals,axis=1)
locs_df = locs_df.explode("intervals")[[0,1,2,11,"intervals"]]
locs_df[["count","intervals"]] = locs_df["intervals"].str.split("_",n=1, expand=True)
locs_df["name"] = locs_df[11] + "_" + locs_df["count"]
# locs_df

In [120]:
locs_df.to_csv("intervals.csv")

In [8]:
file_base = CHR_FILE_PREFIX + str(CHROM) +CHR_FILE_SUFFIX
bim_path = file_base + ".bim"
bed_path = file_base + ".bed"
fam_path = file_base + ".fam"
G = pdp.read_plink1_bin(bed_path,bim_path,fam_path,verbose=False)


In [12]:
bim = G[0]

In [16]:
G1 = bim[bim["chrom"] == 1]

In [20]:
bim[bim["pos"] > 10504]

<xarray.DataArray 'genotype' (variant: 6825471)>
dask.array<getitem, shape=(6825471,), dtype=float32, chunksize=(1024,), chunktype=numpy.ndarray>
Coordinates:
    sample   <U7 '1688461'
  * variant  (variant) <U14 'variant1' 'variant2' ... 'variant6825471'
    fid      object '1688461'
    iid      object '1688461'
    father   object '0'
    mother   object '0'
    gender   object '1'
    trait    object '-9'
    chrom    (variant) object '1' '1' '1' '1' '1' '1' ... '1' '1' '1' '1' '1'
    snp      (variant) object 'rs548419688' 'rs568405545' ... 'rs555533531'
    cm       (variant) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    pos      (variant) int32 10505 10506 10511 ... 249239803 249239825 249239837
    a0       (variant) object 'T' 'G' 'A' 'A' 'T' 'A' ... 'T' 'C' 'C' 'T' 'C'
    a1       (variant) object 'A' 'C' 'G' 'C' 'C' 'C' ... 'G' 'G' 'G' 'G' 'G'